In [1]:
import numpy as np
import pandas as pd
import pickle
import random
# SQL related packages
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
# sklearn packages
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import metrics, preprocessing
from sklearn.linear_model import LogisticRegression as Log
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.metrics.scorer import make_scorer
# text analysis packages
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

#### Load the non-text data

In [2]:
# read the data with the non-text features
patents = pd.read_pickle("patent_data/nontext_features_addwordcounts.p")

In [3]:
print patents.shape
patents.head()

(12030, 21)


,id,title,publication_year,B,C,D,E,F,G,H,...,num_patent_citations,num_nonpatent_citations,num_claims,num_similar_doc,num_authors,payment_times,words_title,words_abstract,words_description,words_claims
0,US6699658B1,Yeast cell surface display of proteins and use...,2004,0,1,0,0,0,0,0,...,28,34,42,1,4,3,10,207,519781,954
1,US6699724B1,Metal nanoshells for biosensing applications,2004,0,0,0,0,0,1,0,...,47,44,25,0,4,3,6,179,71698,407
2,US6690816B2,Systems and methods for tubular object process...,2004,0,0,0,0,0,1,0,...,9,0,32,1,4,1,8,114,122503,1105
3,US6711436B1,"Compositions, apparatus and methods for facili...",2004,0,0,0,0,0,0,0,...,105,109,45,7,1,3,9,290,316007,196
4,US6711432B1,Computer-aided orthopedic surgery,2004,0,0,0,0,0,0,0,...,15,114,44,3,4,3,4,139,49083,683


#### Format the data

In [4]:
# reformat the response variable into binary
y_data = np.zeros(patents.shape[0])
y_data[patents['payment_times'].values >= 2] = 1

print "Percentage of patents with > 1 maintenance fee payments: ", np.mean(y_data)

Percentage of patents with > 1 maintenance fee payments:  0.62859517872


In [5]:
# predictors
x_data = patents.drop(['id', 'title', 'payment_times', 'publication_year'], axis = 1).values
x_data.shape

(12030, 17)

#### Load claims data

In [6]:
#access to sql database
dbname = 'patent_db'
username = 'jy'
pswd = 'jy'

engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))

# reading from sql database
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [7]:
# read data from 2004-2007
years = np.arange(2004, 2008)

# dataframe to store the results
claims = pd.DataFrame()

# import the abstract from each table
for year in years:
    # query:
    sql_query = """
    SELECT claims, id, payment_times, classification
        FROM patents_%s;
    """ %str(year)

    results = pd.read_sql_query(sql_query,con)
    
    claims = pd.concat([claims, results], axis = 0)
    
# check size of the data
claims.shape

(12033, 4)

In [8]:
# remove the patents missing classification data
missing_class_index = (claims['classification'].values == 'NA')

# reassign patent index
claims.index = range(len(claims.index))
# drop the rows
claims =  claims.drop(patents.index[missing_class_index])
claims.shape

/home/jy/anaconda2/envs/my_projects_env/lib/python2.7/site-packages/pandas/indexes/base.py:1434: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 12030 but corresponding boolean dimension is 12033
  result = getitem(key)


(12030, 4)

In [9]:
def tokenize_cleaning(text):
    # tokenize the text first
    tokens = word_tokenize(text.decode('utf-8'))
    
    # lowercase all the words
    tokens = [w.lower() for w in tokens]
    
    # clean up stop words and punctuations 
    stop_list = stopwords.words('english') + list(string.punctuation)

    tokens_no_stop = [token for token in tokens
                        if token not in stop_list]            
    
#     # extract stem of the words
#     stemmer = PorterStemmer()
#     tokens_stem = [stemmer.stem(token) for token in tokens_no_stop]

    # use lemma instead
    # reason: remove the influence of plural or tense
    # but retain the subtle difference in legal writting
    lemmatizer = WordNetLemmatizer()
    tokens_lemma = [lemmatizer.lemmatize(token) for token in tokens_no_stop]
    
    # remove numbers (the actual values are not useful)
    tokens_no_num = []
    for token in tokens_lemma:
        try:
            float(token)
        except:
            tokens_no_num.append(token)
    
    return tokens_no_num

In [10]:
# tokenize_clean the claims and count the occurence of the words

cleaned_text = []
for i in range(claims.shape[0]):
    tokens = tokenize_cleaning(claims['claims'].iloc[i])
    cleaned_text.append(' '.join(word for word in tokens))

In [11]:
# convert to bag-of-words
# min number selected by examining the low-frequency words
vectorizer = CountVectorizer(max_df = 0.4, min_df=5)

# perform a count-based vectorization of the document
word_vec = vectorizer.fit(cleaned_text)
word_counts = word_vec.transform(cleaned_text)

# convert to array
word_counts = word_counts.toarray()
word_counts.shape

(12030, 8035)

In [12]:
# still need to remove some number words 
# due to how CountVectorizer treats '-' and '/'

# remove any word with numbers in it
words = word_vec.get_feature_names()
num_word_index = np.zeros(len(words))

for i in range(len(words)):
    word = words[i]
    for j in range(len(word)):
        try:
            float(word[j])
            num_word_index[i] = 1
            break
        except:
            continue
        
print "Number of numerical words detected: ", int(np.sum(num_word_index))

# remove the number words
words_no_num = np.asarray(words)[num_word_index == 0]
word_counts = word_counts[:, num_word_index == 0]

word_counts.shape

Number of numerical words detected:  222


(12030, 7813)

In [13]:
# combine the claims data with the non-text features
x_data = np.concatenate([x_data, word_counts], axis = 1)

In [14]:
# split train and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, 
                                                    test_size = 2000, 
                                                    random_state = 123)

print "Dataset dimensions:"
print "x_train: ", x_train.shape
print "x_test: ", x_test.shape
print "y_train: ", y_train.shape
print "y_test: ", y_test.shape

Dataset dimensions:
x_train:  (10030, 7830)
x_test:  (2000, 7830)
y_train:  (10030,)
y_test:  (2000,)


### Using subsampling to generate balanced training data

In [15]:
### subsampling the training data
# sample the same number of'useful' patents as the 'not useful' patents
# size of each class
num_size = np.sum(y_train == 0)

#random shuffle the rows
n = x_train.shape[0]
perm = range(n)
np.random.shuffle(perm)

x_train = x_train[perm]
y_train = y_train[perm]

# separate the two classes
x_useful = x_train[y_train == 1, :]
x_not_useful = x_train[y_train == 0, :]
y_useful = y_train[y_train == 1]
y_not_useful = y_train[y_train == 0]

# sample num_size from the 'useful' class
x_useful = x_useful[:num_size]
y_useful = y_useful[:num_size]

# combine the two classes
x_train_sub = np.concatenate((x_useful, x_not_useful), axis = 0)
y_train_sub = np.concatenate((y_useful, y_not_useful), axis = 0)

# shuffle again
# shuffle the combined data
n2 = x_train_sub.shape[0]
perm2 = range(n2)
np.random.shuffle(perm2)

x_train_sub = x_train_sub[perm2]
y_train_sub = y_train_sub[perm2]

# check the size
print x_train_sub.shape
print y_train_sub.shape

(7468, 7830)
(7468,)


In [16]:
# standardize the predictors
scaler = preprocessing.StandardScaler()

x_train_std = scaler.fit_transform(x_train_sub)
x_test_std = scaler.transform(x_test)

/home/jy/anaconda2/envs/my_projects_env/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


#### PCA on the claims word-counts

In [17]:
# try dimensionality reduction using PCA
pca = PCA()

x_train_counts_pca = pca.fit_transform(x_train_std[:, 17:])
x_test_counts_pca = pca.transform(x_test_std[:, 17:])

In [18]:
# find the cum-variance explained at each level
total_var = np.cumsum(pca.explained_variance_ratio_)
n_pc = np.where((total_var > 0.9) == True)[0][0]

print "The number of PCs that can explain > 90% variability: ", n_pc

The number of PCs that can explain > 90% variability:  2481


In [19]:
# combine to the non-text features
x_train_pca = np.concatenate([x_train_std[:, :17], x_train_counts_pca[:, :n_pc]], axis = 1)
x_test_pca = np.concatenate([x_test_std[:, :17], x_test_counts_pca[:, :n_pc]], axis = 1)

### Model training

In [23]:
# write my own scoring function
def my_loss_func(y_actual, y_pred):
    
    # set the price of applying for each patent
    patent_cost = 10
    
    # set the value of a useful patent
    useful_value = 15
    
    total_cost = np.sum(y_pred == 1) * patent_cost
    total_value = np.sum((y_actual == 1) & (y_pred == 1)) * useful_value
    
    profit = total_value -  total_cost
    
    return profit

my_scorer = make_scorer(my_loss_func, greater_is_better=True)

#### Logistic Regression

In [21]:
### use logistic regression

# call the model function
model = Log()
# parameter tuning
c =  np.logspace(-5, 5, 11)

# use grid search with 5-fold CV
grid_model = GridSearchCV(model, param_grid = {'C': c}, cv  = 5, scoring = my_scorer)
# fit on the data
grid_model.fit(x_train_pca, y_train_sub) 

In [27]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.564184495693
Best parameter:  {'C': 0.0001}


In [29]:
# check confusion matrix
best_log = grid_model.best_estimator_
best_log.fit(x_train_pca, y_train_sub)

y_pred = best_log.predict(x_test_pca)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_pred, y_test)
print "Precision: ", metrics.precision_score(y_pred, y_test)
print "Recall: ", metrics.recall_score(y_pred, y_test)
print "Net value: ", my_loss_func(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.536575228595
F1 score:  0.552747693542
Precision:  0.720711297071
Recall:  0.448275862069


array([[602, 267],
       [848, 689]])

In [48]:
model = Log()

# parameter tuning
c =  np.logspace(-5, 3, 9)

# add class weight tuning to the random forest model
weights = np.logspace(-4,0,5)
weight_list_dict = [{0:1, 1: weights[i]} for i in range(len(weights))]

# use grid search with 5-fold CV
grid_model = GridSearchCV(model, param_grid = {'C': c,'class_weight': weight_list_dict}, 
                          cv  = 5, scoring = 'f1')
# fit on the data
grid_model = grid_model.fit(x_train_pca, y_train_sub)

In [49]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.528970792818
Best parameter:  {'C': 1.0, 'class_weight': {0: 1, 1: 1.0}}


In [50]:
# check confusion matrix
best_log = grid_model.best_estimator_
best_log.fit(x_train_pca, y_train_sub)

y_pred = best_log.predict(x_test_pca)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred)
print "Precision: ", metrics.precision_score(y_test, y_pred)
print "Recall: ", metrics.recall_score(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.540315876974
F1 score:  0.586077844311
Precision:  0.68986784141
Recall:  0.509433962264


array([[517, 352],
       [754, 783]])

#### Random Forest

In [24]:
### tune random forest

model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 10, 2)

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space}, 
                          cv  = 5, scoring = my_scorer)
# fit on the data
grid_model = grid_model.fit(x_train_pca, y_train_sub)

In [25]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: -1608.98232458
Best parameter:  {'max_features': 8}


In [26]:
# check confusion matrix
best_rf = grid_model.best_estimator_
best_rf.fit(x_train_pca, y_train_sub)

y_pred = best_rf.predict(x_test_pca)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_pred, y_test)
print "Precision: ", metrics.precision_score(y_pred, y_test)
print "Recall: ", metrics.recall_score(y_pred, y_test)
print "Net value: ", my_loss_func(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.5035
F1 score:  0.558470431303
Precision:  0.496050552923
Recall:  0.638860630722
Net value:  -410


array([[379, 355],
       [638, 628]])

In [27]:
# tuning class weight
model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 10, 2)

# add class weight tuning to the random forest model
weights = np.logspace(-4,0,5)
weight_list_dict = [{0:1, 1: weights[i]} for i in range(len(weights))]

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space,
                                       'class_weight': weight_list_dict}, 
                          cv  = 5, scoring = my_scorer)
# fit on the data
grid_model = grid_model.fit(x_train_pca, y_train_sub)

In [28]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: -1584.99196572
Best parameter:  {'max_features': 8, 'class_weight': {0: 1, 1: 1.0}}


In [30]:
# check confusion matrix
best_rf = grid_model.best_estimator_
best_rf.fit(x_train_pca, y_train_sub)

y_pred = best_rf.predict(x_test_pca)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_pred, y_test)
print "Precision: ", metrics.precision_score(y_pred, y_test)
print "Recall: ", metrics.recall_score(y_pred, y_test)
print "Net value: ", my_loss_func(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.5035
F1 score:  0.556894243641
Precision:  0.492890995261
Recall:  0.64
Net value:  -390


array([[383, 351],
       [642, 624]])

In [54]:
# pickle the rf model
pickle.dump(best_rf, open("models/best_rf_combined.p", 'wb'))

In [55]:
# pickle the pca and scaler
pickle.dump(pca, open("models/best_rf_pca.p", 'wb'))
pickle.dump(scaler, open("models/best_rf_scaler.p", 'wb'))

In [41]:
## try open the model and predict
#rf = pickle.load(open("models/best_rf.p", 'r'))

### Directly work on the imbalanced training data + tune class weight

In [52]:
# standardize the predictors
scaler = preprocessing.StandardScaler()

x_train_std = scaler.fit_transform(x_train)
x_test_std = scaler.transform(x_test)

In [53]:
# try dimensionality reduction using PCA
pca = PCA()

x_train_counts_pca = pca.fit_transform(x_train_std[:, 13:])
x_test_counts_pca = pca.transform(x_test_std[:, 13:])

In [54]:
# find the cum-variance explained at each level
total_var = np.cumsum(pca.explained_variance_ratio_)
n_pc = np.where((total_var > 0.9) == True)[0][0]

print "The number of PCs that can explain > 90% variability: ", n_pc

The number of PCs that can explain > 90% variability:  2797


In [55]:
# combine to the non-text features
x_train_pca = np.concatenate([x_train_std[:, :13], x_train_counts_pca[:, :n_pc]], axis = 1)
x_test_pca = np.concatenate([x_test_std[:, :13], x_test_counts_pca[:, :n_pc]], axis = 1)

In [56]:
# Random Forest + tuning class weight
model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 8, 2)

# add class weight tuning to the random forest model
weights = np.logspace(0,4,5)
weight_list_dict = [{0:1, 1: weights[i]} for i in range(len(weights))]

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space,
                                       'class_weight': weight_list_dict}, 
                          cv  = 5, scoring = my_scorer)
# fit on the data
grid_model = grid_model.fit(x_train_pca, y_train)

In [57]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 16588.0070657
Best parameter:  {'max_features': 2, 'class_weight': {0: 1, 1: 1.0}}


In [60]:
# check confusion matrix
best_rf = grid_model.best_estimator_
best_rf.fit(x_train_pca, y_train)

y_pred = best_rf.predict(x_test_pca)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred)
print "Precision: ", metrics.precision_score(y_test, y_pred)
print "Recall: ", metrics.recall_score(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

 Test accuracy:  0.63757273483
F1 score:  0.770042194093
Precision:  0.647450110865
Recall:  0.949902407287


array([[  74,  795],
       [  77, 1460]])